In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
# Загрузка данных
data = pd.DataFrame({'timestamp': [...], 'buttonKey': [...], 'isWorkingMode': [...]})

# Преобразование временной метрики
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

# Преобразование 'isWorkingMode' в числа
label_encoder = LabelEncoder()
data['isWorkingMode'] = label_encoder.fit_transform(data['isWorkingMode'])

# Создание последовательностей
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i+seq_length]
        sequences.append((seq[['buttonKey', 'isWorkingMode']].values, data['isWorkingMode'].iloc[i + seq_length]))
    return sequences

seq_length = 10  # Например, последовательность из 10 нажатий
sequences = create_sequences(data, seq_length)

X = np.array([seq[0] for seq in sequences])
y = np.array([seq[1] for seq in sequences])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Для бинарной классификации

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


In [ ]:
def predict_next_state(new_data):
    # Предобработка нового ввода аналогично обучающим данным
    new_seq = np.array(new_data).reshape((1, seq_length, 2))  # учтите размерности
    prediction = model.predict(new_seq)
    return prediction[0][0] > 0.5  # Порог для бинарной классификации
